# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [1]:
!uv pip install openai
!uv pip install python-dotenv
!uv pip install tiktoken

Using Python 3.11.10 environment at: /Users/ryuichi/.venv
Audited 1 package in 14ms
Using Python 3.11.10 environment at: /Users/ryuichi/.venv
Audited 1 package in 2ms
Using Python 3.11.10 environment at: /Users/ryuichi/.venv
Audited 1 package in 3ms


In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# .envファイルをロードして環境変数を読み込む
load_dotenv()

# 環境変数から値を取得
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# 必須の環境変数が欠けている場合エラーをスロー
if not azure_endpoint or not api_key or not api_version:
    raise ValueError("必須の環境変数の値が取得できていません。環境変数を確認してください。")

# Azure OpenAI Clientの初期化
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

In [3]:
import textwrap
import tiktoken # tiktoken ライブラリをインポート

def fetch_top_tokens_and_probabilities(payload, system_message="あなたは優秀な映画の専門家です。"):
    """
    Azure OpenAIのモデルに対してプロンプトに続くトークンとその確率を取得します。

    Parameters:
        - payload (dict): モデル名、プロンプト、温度、top_kを含む辞書。
        - system_message (str): システムメッセージ（デフォルト値あり）。

    Returns:
        tuple: (APIが実際に生成した最初のトークン文字列, 上位トークンとその確率のリスト)
    """
    try:
        # API呼び出し: 次のトークン予測
        # logprobs=True に加えて、top_logprobs パラメータを追加
        response = client.chat.completions.create(
            model=payload["model"],
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": payload["prompt"]}
            ],
            temperature=payload["temperature"],
            max_tokens=1, # 続く「一つの」トークンを見るため max_tokens=1
            logprobs=True,
            top_logprobs=payload["top_k"] # 返却する上位トークン数を指定
        )

        # print(f"APIレスポンス: {response}") # デバッグが必要な場合のみ表示推奨

        response_dict = response.model_dump()

        # max_tokens=1 なので、生成された最初のトークンを取得
        generated_token_text = response_dict["choices"][0]["message"]["content"]

        # 上位トークンとその確率を取得
        # logprobs -> content -> [0] (最初の出力トークン) -> top_logprobs にアクセス
        # top_logprobs はリスト内の辞書です: [{'token': ' abc', 'logprob': -0.1, 'bytes': [...]}, ...]
        # content リストが空でないかチェック（max_tokens=1なので通常は1つの要素があるはず）
        if not response_dict["choices"][0].get("logprobs") or not response_dict["choices"][0]["logprobs"].get("content"):
             print("警告: APIレスポンスにlogprobs情報が含まれていませんでした。")
             return generated_token_text, []

        # 最初の生成トークン位置での上位候補トークン情報を取得
        top_logprobs_list_of_dicts = response_dict["choices"][0]["logprobs"]["content"][0]["top_logprobs"]

        if not top_logprobs_list_of_dicts:
             print("警告: APIから上位トークン情報が返されませんでした。top_logprobs=0 の可能性があります。")
             return generated_token_text, []


        top_tokens_and_probabilities = [
            (token_info["token"], round(pow(10, token_info["logprob"]), 5))
            for token_info in top_logprobs_list_of_dicts
        ]

        # APIが top_logprobs で指定した数のトークンを確率付きで返しているので、ソートして返します
        return generated_token_text, sorted(top_tokens_and_probabilities, key=lambda x: x[1], reverse=True)

    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました: {e}")
        # エラー発生時は生成されたトークンも確率リストも取得できない
        return None, None

payload = {
    "model": "gpt-4o",  # あなたのAzure OpenAIでデプロイしたモデルのデプロイメント名を指定
    # "prompt": "The movie was full of",
    "prompt": "The movie was full of",
    "temperature": 0.7, # Temperature affects sampling of the *generated* token, but logprobs shows potential options regardless.
    "top_k": 10 # APIに返すように要求する上位トークンの数
}

print(f"「'{payload['prompt']}'」に続くトークン上位{payload['top_k']}個と、その確率（尤度）を求めよ。")
print(f"ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。")

# --- プロンプトのトークン化を表示 ---
print("\n--- プロンプトのトークン化結果 ---")
try:
    # モデル名に対応するエンコーディングを取得
    encoding = tiktoken.encoding_for_model(payload["model"])

    # プロンプトをトークンIDのリストにエンコード
    input_token_ids = encoding.encode(payload["prompt"])

    # 各トークンIDを元のテキストに戻す（デコード）
    # 各IDごとにデコードすると、元の区切り（スペースなど）を維持しやすい
    input_tokens_text = [encoding.decode([token_id]) for token_id in input_token_ids]

    print(f"元のプロンプト: '{payload['prompt']}'")
    print(f"トークンID列: {input_token_ids}")
    print(f"トークンテキスト列: {input_tokens_text}")
    print(f"トークン数: {len(input_token_ids)}")

except Exception as e:
    print(f"プロンプトのトークン化中にエラーが発生しました: {e}")
    print("tiktoken がインストールされているか確認してください (`pip install tiktoken`)。")


# --- 続くトークンの確率を取得 ---
print("\n--- 続くトークン候補と確率 ---")
# APIを呼び出して続くトークンの確率を取得
generated_token, top_tokens_and_probabilities = fetch_top_tokens_and_probabilities(payload)

# 結果を表示
if generated_token is not None and top_tokens_and_probabilities is not None:
    # API呼び出しによって実際に生成された最初のトークン
    # これは logprobs リストに含まれる可能性が高いですが、サンプリングによる結果です。
    print(f"API呼び出しで実際に生成された最初のトークン (max_tokens=1 の結果): '{generated_token}'")

    print(f"\n'{payload['prompt']}' に続く位置でのトークン候補上位{payload['top_k']}件とその確率:")
    if top_tokens_and_probabilities:
        # 確率の高い順に表示
        for token, probability in top_tokens_and_probabilities:
            print(f"  '{token}': {probability}") # トークンを引用符で囲むと見やすい
    else:
        print("上位トークン候補情報は取得できませんでした。")

「'The movie was full of'」に続くトークン上位10個と、その確率（尤度）を求めよ。
ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

--- プロンプトのトークン化結果 ---
元のプロンプト: 'The movie was full of'
トークンID列: [976, 8249, 673, 3149, 328]
トークンテキスト列: ['The', ' movie', ' was', ' full', ' of']
トークン数: 5

--- 続くトークン候補と確率 ---
API呼び出しで実際に生成された最初のトークン (max_tokens=1 の結果): 'The'

'The movie was full of' に続く位置でのトークン候補上位10件とその確率:
  'The': 0.17045
  'It': 0.00539
  'Could': 0.00303
  'intr': 0.00096
  'st': 0.00054
  'em': 0.0003
  'unexpected': 0.0003
  'Can': 0.00017
  'v': 0.0001
  'spect': 0.0001


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [4]:
import textwrap
import tiktoken
import os # 環境変数からAPIキーなどを読み込む場合に使用

# --- テキスト補完生成関数 ---
def generate_text_completions(client, model, prompt, max_tokens, temperature, top_p, n):
    """
    指定されたパラメータでAzure OpenAIモデルから複数のテキスト補完を生成します。

    Parameters:
        - client: AzureOpenAI クライアントインスタンス
        - model (str): モデル名
        - prompt (str): 入力プロンプト
        - max_tokens (int): 生成する最大トークン数 (1以上)
        - temperature (float): サンプリング温度 (0.0-2.0)。0で決定論的。
        - top_p (float): top_p サンプリング確率 (0.0-1.0)。1.0でtemperatureサンプリングに近い。
        - n (int): 生成する補完の数 (1以上)

    Returns:
        list: 生成されたテキスト文字列のリスト。エラー発生時はエラーメッセージを含むリスト。
    """
    if client is None:
        return ["Error: Azure OpenAI client is not initialized."] * n

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            n=n, # 複数の補完を生成
            # logprobs, top_logprobs はここでは使用しない
        )

        # 生成されたテキストをリストとして抽出
        completions = [choice.message.content for choice in response.choices]

        return completions

    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました (温度={temperature}, top_p={top_p}): {e}")
        return [f"Error generating completion: {e}"] * n # エラー時はエラーメッセージを含むリストを返す

# --- 主処理 ---

model_name = "gpt-4o"
prompt_text = "The movie was full of"
generate_max_tokens = 100 # 生成するテキストの最大長（トークン単位）。必要に応じて調整。

# 試行する温度とtop_pの値のリスト
# 温度 (Temperature): 高いほどランダム性が増す (多様だが脱線も)。低いほど決定論的 (毎回同じ傾向)。
temperatures_to_test = [0.0, 0.7, 1.0, 1.5]
# top_p (Nucleus Sampling): 累積確率が top_p になるまで確率の高いトークン候補を選び、その中からサンプリング。
# 低いほど候補が絞られ予測可能に。高いほど候補が増え多様に (1.0 は temperature サンプリングに近い)。
top_p_to_test = [0.1, 0.5, 1.0]

# 各設定で生成するテキストの数
num_completions_per_setting = 3 # 各温度・top_pの組み合わせで3つの補完を生成

print(f"=== 課題 ===")
print(f"「'{prompt_text}'」に続くテキストを複数予測し、デコーディング方法（温度、top_p）による変化を観察する。")
print(f"モデル: {model_name}")
print(f"生成最大トークン数: {generate_max_tokens}")
print("============")

print("\n=== テキスト補完の生成と観察 ===")

# 温度とtop_pの組み合わせごとに補完を生成
for temp in temperatures_to_test:
    for p in top_p_to_test:
        # temperature=0.0 の場合、top_p は通常無視され決定論的なサンプリングになります。
        # 重複を避けるため、temperature=0.0 の場合は top_p=1.0 の設定のみ実行します。
        if temp == 0.0 and p != 1.0:
            continue # 他の top_p はスキップ

        # 設定の表示
        if temp == 0.0:
             print(f"\n--- 設定: 温度={temp} (決定論的サンプリング) ---")
        else:
             print(f"\n--- 設定: 温度={temp}, top_p={p} ---")

        # テキスト補完を生成
        completions = generate_text_completions(
            client, # クライアントオブジェクトを渡す
            model_name,
            prompt_text,
            generate_max_tokens,
            temp,
            p,
            num_completions_per_setting
        )

        # 生成された補完を表示
        for i, text in enumerate(completions):
            print(f"  補完 {i+1}:")
            # テキストを整形して表示
            print(textwrap.fill(text, width=80, initial_indent='    ', subsequent_indent='    '))

=== 課題 ===
「'The movie was full of'」に続くテキストを複数予測し、デコーディング方法（温度、top_p）による変化を観察する。
モデル: gpt-4o
生成最大トークン数: 100

=== テキスト補完の生成と観察 ===

--- 設定: 温度=0.0 (決定論的サンプリング) ---
  補完 1:
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kept the audience engaged from start to finish.
    Whether it was the gripping storyline, the powerful performances, or the
    breathtaking cinematography, it delivered an unforgettable experience.
  補完 2:
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kept the audience engaged from start to finish.
    Whether it was the gripping storyline, the powerful performances, or the
    breathtaking cinematography, it delivered an unforgettable experience.
  補完 3:
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kept the audience engaged from start to finish.
    Whether it was the gripping storyline, the powerf

## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [5]:
import textwrap
import tiktoken
import os
import math # logprob（対数確率）を通常の確率に変換するためにmathモジュールを使用

# --- テキスト生成と確率取得関数 ---
def generate_text_with_probabilities(client, model, prompt, max_tokens):
    """
    プロンプトに続くテキストを生成し、各生成トークンの確率を返します。

    Parameters:
        - client: 初期化済みの AzureOpenAI クライアントインスタンス
        - model (str): Azureでデプロイしたモデルのデプロイメント名
        - prompt (str): 入力プロンプト
        - max_tokens (int): 生成する最大トークン数 (1以上)

    Returns:
        tuple: (生成されたテキスト文字列, 各トークンと確率のリスト)。
               エラー発生時は (None, None)。
    """
    if client is None:
        print("エラー: OpenAI client is not initialized. Cannot make API call.")
        return None, None

    try:
        # API呼び出し: テキスト生成とトークン確率の取得
        # temperature は 0.0 (決定論的) を推奨 - 最も尤もらしいシーケンスが生成される
        # n=1 で単一の補完を取得
        # logprobs=True で生成トークンそれぞれの確率を要求
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.0, # 確率を観察しやすくするため決定論的にする
            n=1,             # 1つの補完を取得
            logprobs=True    # 生成トークンの確率情報を取得
            # top_p, top_logprobs は今回は使用しない
        )

        # print(f"APIレスポンス: {response}") # 必要に応じてデバッグ用にコメント解除

        response_dict = response.model_dump()

        # 生成されたテキスト全体を取得 (choices[0] は n=1 なので最初の要素)
        generated_text = response_dict["choices"][0]["message"]["content"]

        # logprobs 情報があるか確認
        # logprobs=True にしても、生成テキストがない場合など content が None になる可能性あり
        logprobs_info = response_dict["choices"][0].get("logprobs")
        if not logprobs_info or not logprobs_info.get("content"):
             print("警告: APIレスポンスにlogprobs情報が含まれていませんでした。")
             # テキスト自体は返せたかもしれないが、確率は不明
             return generated_text, []

        # 生成された各トークンとそのlogprobを取得
        token_logprob_list = logprobs_info["content"]

        # 各トークンと確率のリストを作成
        token_probability_list = []
        for token_info in token_logprob_list:
             token_text = token_info["token"]
             logprob = token_info["logprob"]
             # logprob (対数確率) を通常の確率に変換
             # OpenAI API の logprob は自然対数 (底 e) です
             probability = math.exp(logprob)
             # 確率を小数点以下5桁で丸める
             token_probability_list.append((token_text, round(probability, 5)))

        return generated_text, token_probability_list

    except Exception as e:
        print(f"API呼び出し中にエラーが発生しました: {e}")
        # エラー発生時はテキストも確率も取得できていない
        return None, None

# --- 主処理 ---

model_name = "gpt-4o" # あなたのAzure OpenAIでデプロイしたモデルのデプロイメント名を指定
prompt_text = "The movie was full of"
generate_max_tokens_limit = 30 # 生成を打ち切る最大トークン数（適当な長さ）

print(f"=== 課題 ===")
print(f"「'{prompt_text}'」に続くテキストを予測し、生成された各トークンの尤度を表示する。")
print(f"モデル: {model_name}")
print(f"生成最大トークン数: {generate_max_tokens_limit}")
print("============")

# --- 元のプロンプトのトークン化を表示 (参考情報として維持) ---
print("\n--- 元のプロンプトのトークン化結果 ---")
# tiktoken は API クライアント初期化の成功とは独立して動作
# ただし、モデル名がtiktokenでサポートされている必要がある
try:
    encoding = tiktoken.encoding_for_model(model_name)
    input_token_ids = encoding.encode(prompt_text)
    input_tokens_text = [encoding.decode([token_id]) for token_id in input_token_ids]

    print(f"元のプロンプト: '{prompt_text}'")
    print(f"トークンID列: {input_token_ids}")
    print(f"トークンテキスト列: {input_tokens_text}")
    print(f"トークン数: {len(input_token_ids)}")

except Exception as e:
    print(f"プロンプトのトークン化中にエラーが発生しました: {e}")
    print("モデル名が正しいか、tiktoken がインストールされているか確認してください (`pip install tiktoken`)。")


print("\n=== 生成されたテキストとそのトークン確率 ===")

# テキストを生成し、各トークンの確率を取得
# generate_text_with_probabilities 関数内でクライアント初期化の成功をチェック
generated_text, token_probabilities = generate_text_with_probabilities(
    client, # 初期化されたクライアントを渡す
    model_name,
    prompt_text,
    generate_max_tokens_limit
)

# 結果を表示
if generated_text is not None:
    print(f"\n生成されたテキスト (最大{generate_max_tokens_limit}トークン):")
    # 生成テキスト全体を表示
    print(textwrap.fill(generated_text, width=80, initial_indent='    ', subsequent_indent='    '))

    if token_probabilities:
        print("\n各生成トークンとその確率:")
        # 各トークンと確率を表示
        # 見やすくするために、トークンを repr() で囲み、確率を調整して表示
        # repr() を使うと '\n', '\t', ' ' (スペース) など、トークンの厳密な内容が見える
        for token, probability in token_probabilities:
             display_token = repr(token)
             # 表示を揃えるために幅を指定 (必要に応じて調整)
             print(f"  トークン: {display_token:<15}, 確率: {probability}")
    else:
        print("\nトークン確率情報は取得できませんでした。（API呼び出しエラーまたはlogprobs情報なし）")

=== 課題 ===
「'The movie was full of'」に続くテキストを予測し、生成された各トークンの尤度を表示する。
モデル: gpt-4o
生成最大トークン数: 30

--- 元のプロンプトのトークン化結果 ---
元のプロンプト: 'The movie was full of'
トークンID列: [976, 8249, 673, 3149, 328]
トークンテキスト列: ['The', ' movie', ' was', ' full', ' of']
トークン数: 5

=== 生成されたテキストとそのトークン確率 ===

生成されたテキスト (最大30トークン):
    The movie was full of **unexpected twists**, **emotional depth**, and
    **stunning visuals** that kept the audience engaged from start to finish.

各生成トークンとその確率:
  トークン: 'The'          , 確率: 0.29954
  トークン: ' movie'       , 確率: 0.99663
  トークン: ' was'         , 確率: 0.99999
  トークン: ' full'        , 確率: 1.0
  トークン: ' of'          , 確率: 1.0
  トークン: ' **'          , 確率: 0.54964
  トークン: 'unexpected'   , 確率: 0.51165
  トークン: ' twists'      , 確率: 0.99525
  トークン: '**,'          , 確率: 0.36773
  トークン: ' **'          , 確率: 0.96097
  トークン: 'em'           , 確率: 0.56557
  トークン: 'otional'      , 確率: 0.99946
  トークン: ' depth'       , 確率: 0.85965
  トークン: '**,'          , 確率: 0.99997
  トークン: ' and'       

## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [6]:
import textwrap
import tiktoken
import os
import math
import time


# --- パープレキシティ計算関数 ---
# top_k_for_logprobs のデフォルト値を 20 に修正
def calculate_perplexity(client, model, sentence, top_k_for_logprobs=20, delay_seconds=0.05):
    """
    指定された文のパープレキシティを計算します。
    文中の各トークンについて、その前のトークン列をプロンプトとしてAPIを呼び出し、
    当該トークンの確率を取得します。

    Parameters:
        - client: 初期化済みの AzureOpenAI クライアントインスタンス
        - model (str): Azureでデプロイしたモデルのデプロイメント名
        - sentence (str): パープレキシティを測定する文
        - top_k_for_logprobs (int): APIで取得する上位トークン確率の数。
                                   モデルの制限により最大値は通常20です。
                                   対象トークンがこの中に含まれる必要があります。
        - delay_seconds (float): API呼び出し間の待ち時間（秒）。レート制限対策。

    Returns:
        float: 計算されたパープレキシティ。計算できない場合は float('inf')。
    """
    # APIの top_logprobs の制限値を確認し、指定値がそれを超えていないかチェック（念のため）
    actual_top_k = min(top_k_for_logprobs, 20) # API制限を考慮
    if top_k_for_logprobs > 20:
         print(f"  警告: 指定された top_k_for_logprobs ({top_k_for_logprobs}) はモデルの上限 (20) を超えています。実際には {actual_top_k} を使用します。")


    if client is None:
        print("エラー: OpenAI client is not initialized. Cannot calculate perplexity.")
        return float('inf')

    try:
        encoding = tiktoken.encoding_for_model(model)
        # 文全体をトークンIDに変換
        sentence_token_ids = encoding.encode(sentence)
        num_tokens = len(sentence_token_ids)

        # 1トークン以下の文はパープレキシティ計算が定義されない（あるいは1となる）
        if num_tokens <= 1:
            print(f"警告: 文 '{sentence}' のトークン数が1以下です ({num_tokens}トークン)。パープレキシティは定義されません。")
            return 1.0 if num_tokens == 1 else float('inf') # 1トークンの場合は通常1、0トークンは無限大

        total_logprob = 0.0
        num_terms = 0 # logprobを合計できた項の数

        print(f"  '{sentence}' ({num_tokens}トークン) のパープレキシティを計算中...")

        # 文の2番目のトークンから最後までループ (w_i | w_1...w_{i-1}) を取得するため
        # ループ変数 i は、sentence_token_ids リストにおける次のトークンのインデックス
        # プロンプトは sentence_token_ids[:i] となり、次のトークンは sentence_token_ids[i]
        # i=1 から num_tokens-1 まで回す (sentence_token_ids[1] から sentence_token_ids[num_tokens-1] まで)
        for i in range(1, num_tokens):
            target_token_id = sentence_token_ids[i]
            # 対象トークンをテキストにデコード (APIレスポンスのトークンと比較するため)
            target_token_text = encoding.decode([target_token_id])

            # プロンプトとして使用するトークン列 (対象トークンの直前まで)
            prompt_token_ids = sentence_token_ids[:i]
            prompt_text = encoding.decode(prompt_token_ids)

            # プロンプトが空になる場合（文頭のトークンに対する確率）は、
            # APIのlogprobs機能では直接取得が難しいため、2番目のトークンから開始しています。
            # パープレキシティの定義によっては最初のトークンの確率も考慮しますが、
            # 一般的なLM評価では2番目以降で計算することが多いです。
            # ここでは i=1 から開始するため、最初のトークン(i=0)をプロンプトにした次のトークン(i=1)から評価します。
            if not prompt_text: # 最初のトークンに対する評価の場合（i=0の時だが、ここではi=1から開始）
                 # print(f"  スキップ: 最初のトークン '{target_token_text}' の確率計算はスキップされます。")
                 # num_terms += 1 # 項数に含める場合はコメント解除（分母に影響）
                 continue # ここでは i=1 からループしているので、i=0 のケースは発生しない


            # API呼び出し: プロンプトに続く最初の1トークンの確率を取得
            # temperature=0.0 で最も尤もらしい分布を取得しやすくする
            # top_logprobs で対象トークンが分布に含まれる可能性を高める（ただし上限あり）
            try:
                api_response = client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "user", "content": prompt_text}
                    ],
                    max_tokens=1, # 次の1トークンのみ予測
                    temperature=0.0, # サンプリングのランダム性をなくす（確率分布は変わらない）
                    logprobs=True, # ログ確率を要求
                    top_logprobs=actual_top_k # 実際の制限値を適用
                )

                # API呼び出し間に遅延を入れる（レート制限対策）
                time.sleep(delay_seconds)

            except Exception as e:
                print(f"  エラー: API呼び出し失敗 (プロンプト: '{prompt_text}'): {e}")
                # API呼び出しに失敗した場合は、計算不可として無限大を返す
                return float('inf')


            # レスポンスからログ確率情報を抽出
            logprobs_info = api_response.choices[0].logprobs
            # logprobs_info.content は生成トークンに関するリスト。max_tokens=1なので通常は要素が1つ。
            # logprobs_info.content[0].top_logprobs が上位候補リスト。
            # top_logprobs_list の各要素は TopLogprob オブジェクトであることに注意
            if not logprobs_info or not logprobs_info.content or not logprobs_info.content[0].top_logprobs:
                print(f"  警告: APIレスポンスに十分な logprobs 情報が含まれていませんでした (プロンプト: '{prompt_text}')。")
                 # logprob 情報がなければ計算不可
                return float('inf')


            top_logprobs_list = logprobs_info.content[0].top_logprobs

            # 上位候補リストから、対象トークンの logprob を探す
            found_logprob = None
            # APIレスポンスのトークンテキストと、tiktokenでデコードしたトークンテキストを比較
            # repr() を使うとスペースや改行などの非表示文字を含めて比較できるためより安全
            target_token_repr = repr(target_token_text)

            for token_info in top_logprobs_list:
                 # ここを修正: ドット記法 (.) で属性にアクセスする
                 if repr(token_info.token) == target_token_repr:
                     found_logprob = token_info.logprob # ここを修正
                     break # 見つかったらループを抜ける


            # 対象トークンの logprob が上位K個の中に見つかったか確認
            if found_logprob is not None:
                total_logprob += found_logprob
                num_terms += 1
                # print(f"    Found '{target_token_text}' ({target_token_repr}): {found_logprob}") # デバッグ用
            else:
                # 対象トークンが上位K個に含まれていない場合、その確率は非常に低い
                # ここでは警告を表示し、非常に小さい確率（大きい負のlogprob）を代用する
                fallback_logprob = -20.0 # 代用する大きな負のlogprob
                total_logprob += fallback_logprob
                num_terms += 1
                print(f"  警告: 対象トークン '{target_token_text}' ({target_token_repr}) が上位 {actual_top_k} 候補に見つかりませんでした (プロンプト: '{prompt_text}')。代用logprob ({fallback_logprob}) を使用します。")
                # 見つからなかった時点で計算不可とするなら以下のコメントを解除
                # return float('inf')


        # パープレキシティの計算
        # Perplexity = exp(- (1/N) * Sum(log P))
        # ここで N は合計したlogprobの数 (num_terms)
        if num_terms == 0:
            print("  エラー: logprobを計算できたトークンがありません。")
            return float('inf')

        average_logprob = total_logprob / num_terms
        perplexity = math.exp(-average_logprob)

        return perplexity

    except Exception as e:
        print(f"  致命的なエラーが発生しました: {e}")
        return float('inf')


# --- 主処理 ---

# パープレキシティを測定する文のリスト
sentences_to_test = [
    "The movie was full of surprises",       # 文法的にも意味的にも自然
    "The movies were full of surprises",     # 主語・動詞が複数形で一致、自然
    "The movie were full of surprises",      # 主語が単数、動詞が複数形で不一致 (文法ミス)
    "The movies was full of surprises",      # 主語が複数、動詞が単数形で不一致 (文法ミス)
    "Surprises full of was movie The",       # 単語順序が不自然 (文法ミス)
    "This is a very common sentence",        # 別の自然な文
    "This very is a common sentence",        # 単語順序が不自然 (文法ミス)
    "The quick brown fox jumps over the lazy dog", # より長い標準的な文
]

model_name = "gpt-4o" # あなたのAzure OpenAIでデプロイしたモデル名を指定
# APIの制限に合わせて top_k_for_logprobs を 20 に修正
top_k_for_logprobs = 20
api_delay_seconds = 0.01 # API呼び出し間の短い遅延 (推奨)

print(f"=== 課題 ===")
print(f"与えられた文に対するパープレキシティを測定し、比較する。")
print(f"モデル: {model_name}")
print(f"logprob取得のための上位K (API制限): {top_k_for_logprobs}")
print(f"API遅延: {api_delay_seconds}秒")
print("============")

# クライアントが初期化されているか確認
if client is None:
    print("\nエラー: Azure OpenAI クライアントが初期化されていません。パープレキシティ計算を実行できません。")
else:
    print("\n=== パープレキシティ測定結果 ===")
    # 各文に対してパープレキシティを計算し表示
    for sentence in sentences_to_test:
        # calculate_perplexity 関数内でクライアント初期化の成功をチェック
        perplexity_value = calculate_perplexity(client, model_name, sentence, top_k_for_logprobs, api_delay_seconds)

        if perplexity_value == float('inf'):
            print(f"文: '{sentence}'")
            print(f"パープレキシティ: 計算不可") # エラーメッセージは関数内で出力済み
        else:
            print(f"文: '{sentence}'")
            print(f"パープレキシティ: {perplexity_value:.4f}") # 小数点以下4桁で表示
        print("-" * 30) # 区切り線

=== 課題 ===
与えられた文に対するパープレキシティを測定し、比較する。
モデル: gpt-4o
logprob取得のための上位K (API制限): 20
API遅延: 0.01秒

=== パープレキシティ測定結果 ===
  'The movie was full of surprises' (6トークン) のパープレキシティを計算中...
  警告: 対象トークン ' movie' (' movie') が上位 20 候補に見つかりませんでした (プロンプト: 'The')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' was' (' was') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' full' (' full') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie was')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' of' (' of') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie was full')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' surprises' (' surprises') が上位 20 候補に見つかりませんでした (プロンプト: 'The movie was full of')。代用logprob (-20.0) を使用します。
文: 'The movie was full of surprises'
パープレキシティ: 485165195.4098
------------------------------
  'The movies were full of surprises' (6トークン) のパープレキシティを計算中...
  警告: 対象トークン ' movies' (' movies') が上位 20 候補に見つかりませんでした (プロンプト: 'The')。代用logprob (-20.0) を使用します。
  警告: 対象トークン ' were' (' were') が上位 20 候補に見つかりませんでし

## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [11]:
# 問題89のセルを実行する前に、問題80や問題85でHugging FaceのBERTトークナイザをロード

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup # AutoModelを使用
from sklearn.metrics import accuracy_score
import time
import os
import pandas as pd

# --- 前提となる変数・クラス・関数の定義 (問題87と同様) ---
# tokenizer, train_bert_dataset, dev_bert_dataset, MAX_LENGTH
# BertSST2Dataset クラス

# --- 変数・クラスが現在のセッションに存在するか確認 ---
required_vars_p89 = ['tokenizer', 'train_bert_dataset', 'dev_bert_dataset', 'MAX_LENGTH']
for var_name in required_vars_p89:
    if var_name not in locals() or locals()[var_name] is None:
        print(f"エラー: 前提となる変数 '{var_name}' が定義されていません。")
        raise NameError(f"Variable '{var_name}' is not defined or None.")

if 'BertSST2Dataset' not in locals():
        print(f"エラー: 前提となるクラス 'BertSST2Dataset' が定義されていません。")
        raise NameError(f"Class 'BertSST2Dataset' is not defined.")

# --- ここから問題89の処理 ---

# 1. 新しいアーキテクチャのモデルクラス定義 (平均プーリング)
class BertMeanPoolingClassifier(nn.Module):
    def __init__(self, model_name, num_labels, dropout_rate=0.1):
        super(BertMeanPoolingClassifier, self).__init__()
        # 事前学習済みBERTモデルをロード (分類ヘッドなし)
        self.bert = AutoModel.from_pretrained(model_name)
        # BERTの隠れ層サイズを取得 (例: bert-base-uncased なら 768)
        self.bert_hidden_size = self.bert.config.hidden_size 
        
        self.dropout = nn.Dropout(dropout_rate)
        # 分類用の線形層
        self.classifier = nn.Linear(self.bert_hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        # BERTモデルから最終層の隠れ状態を取得
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state # (batch_size, seq_len, hidden_size)
        
        # 平均プーリング (attention_mask を利用してパディングを無視)
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9) # ゼロ除算を避ける
        mean_pooled_output = sum_embeddings / sum_mask
        # mean_pooled_output: (batch_size, hidden_size)
        
        # ドロップアウトと分類
        pooled_output_dropout = self.dropout(mean_pooled_output)
        logits = self.classifier(pooled_output_dropout) # (batch_size, num_labels)
        
        return logits

# --- モデルと学習の準備 ---
model_name_p89 = "bert-base-uncased" # 問題87と同じ事前学習モデルを使用
num_labels_p89 = 2 # ポジネガ2値分類

# デバイス設定
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU ({torch.cuda.get_device_name(0)}) を使用します。")
else:
    device = torch.device("cpu")
    print("GPUが利用できません。CPUを使用します。")

# モデルのインスタンス化とデバイスへの転送
model_meanpool = BertMeanPoolingClassifier(model_name_p89, num_labels_p89)
model_meanpool.to(device)
print(f"\n平均プーリングを用いたBERT分類モデル '{model_name_p89}' をロードし、デバイスに転送しました。")
print(model_meanpool)


# データローダーの準備 (問題87と同様)
# BertSST2Dataset は {key: val[idx]} を返すので、モデルの入力に合わせて調整が必要な場合がある
# BertMeanPoolingClassifier は input_ids と attention_mask を想定
class BertInputDataset(Dataset): # BertSST2Dataset を少し変更
    def __init__(self, encodings_dict, labels_tensor):
        self.input_ids = encodings_dict['input_ids']
        self.attention_mask = encodings_dict['attention_mask']
        # token_type_ids があればそれも
        self.token_type_ids = encodings_dict.get('token_type_ids', None) # なければNone
        self.labels = labels_tensor

    def __getitem__(self, idx):
        item = {
            'input_ids': self.input_ids[idx].clone().detach(),
            'attention_mask': self.attention_mask[idx].clone().detach()
        }
        if self.token_type_ids is not None:
             item['token_type_ids'] = self.token_type_ids[idx].clone().detach()
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

train_dataset_mp = BertInputDataset(
    {'input_ids': train_bert_dataset['input_ids'], 
     'attention_mask': train_bert_dataset['attention_mask'],
     'token_type_ids': train_bert_dataset.get('token_type_ids', None)}, # token_type_idsも渡す
    train_bert_dataset['labels']
)
dev_dataset_mp = BertInputDataset(
    {'input_ids': dev_bert_dataset['input_ids'], 
     'attention_mask': dev_bert_dataset['attention_mask'],
     'token_type_ids': dev_bert_dataset.get('token_type_ids', None)},
    dev_bert_dataset['labels']
)

batch_size_mp = 16
train_dataloader_mp = DataLoader(train_dataset_mp, batch_size=batch_size_mp, shuffle=True)
dev_dataloader_mp = DataLoader(dev_dataset_mp, batch_size=batch_size_mp, shuffle=False)


# 学習パラメータ
learning_rate_mp = 2e-5
num_epochs_mp = 3 # 問題87と同程度のエポック数で比較

# 損失関数 (クラス数が2なのでCrossEntropyLossが一般的)
# モデルの出力ロジットは (batch_size, num_labels) の形状になる
criterion_mp = nn.CrossEntropyLoss() 
optimizer_mp = optim.AdamW(model_meanpool.parameters(), lr=learning_rate_mp)
total_steps_mp = len(train_dataloader_mp) * num_epochs_mp
scheduler_mp = get_linear_schedule_with_warmup(optimizer_mp, num_warmup_steps=0, num_training_steps=total_steps_mp)

print(f"\n平均プーリングモデルのファインチューニング (バッチサイズ={batch_size_mp}, エポック数={num_epochs_mp}) を開始します...")
start_time_mp = time.time()

# --- 学習ループ (問題87とほぼ同じだが、損失計算の仕方が少し異なる可能性) ---
for epoch in range(num_epochs_mp):
    print(f"\n--- Epoch {epoch+1}/{num_epochs_mp} ---")
    model_meanpool.train()
    total_train_loss = 0
    
    for batch_idx, batch in enumerate(train_dataloader_mp):
        optimizer_mp.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device) # ラベルは (batch_size) の形状で、クラスインデックス (0 or 1)
        # token_type_ids は今回のモデルでは使わないが、渡せるようにしておく
        # token_type_ids = batch.get('token_type_ids', None)
        # if token_type_ids is not None:
        #     token_type_ids = token_type_ids.to(device)
        
        # logits = model_meanpool(input_ids=input_ids, attention_mask=attention_mask)
        # AutoModelは **kwargs を受け付けるので、token_type_ids も渡せる
        model_inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
        # if token_type_ids is not None: # BertMeanPoolingClassifier側で **kwargs を受け取らないので、直接渡せない
        #     model_inputs['token_type_ids'] = token_type_ids
        logits = model_meanpool(**model_inputs) # **model_inputs で展開して渡す

        loss = criterion_mp(logits, labels) # CrossEntropyLossはロジットと整数のラベルを期待
        total_train_loss += loss.item()
        
        loss.backward()
        optimizer_mp.step()
        scheduler_mp.step()
        
        if (batch_idx + 1) % (len(train_dataloader_mp) // 10) == 0 or (batch_idx + 1) == len(train_dataloader_mp):
            print(f"  Batch [{batch_idx+1}/{len(train_dataloader_mp)}], Avg Train Loss so far: {total_train_loss / (batch_idx+1):.4f}")

    avg_train_loss = total_train_loss / len(train_dataloader_mp)
    print(f"Epoch [{epoch+1}/{num_epochs_mp}] 完了, 平均訓練損失: {avg_train_loss:.4f}")

    # エポックごとに検証データで評価
    model_meanpool.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    with torch.no_grad():
        for batch in dev_dataloader_mp:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # token_type_ids = batch.get('token_type_ids', None)
            # if token_type_ids is not None:
            #     token_type_ids = token_type_ids.to(device)
            
            model_inputs_dev = {'input_ids': input_ids, 'attention_mask': attention_mask}
            # if token_type_ids is not None:
            #     model_inputs_dev['token_type_ids'] = token_type_ids
            logits = model_meanpool(**model_inputs_dev)
            
            loss = criterion_mp(logits, labels)
            total_eval_loss += loss.item()
            
            predictions = torch.argmax(logits, dim=-1) # (batch_size) の形状
            total_eval_accuracy += accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy()) * labels.size(0)

    avg_val_accuracy = total_eval_accuracy / len(dev_dataset_mp)
    avg_val_loss = total_eval_loss / len(dev_dataloader_mp)
    print(f"  Epoch [{epoch+1}/{num_epochs_mp}], 検証データ: 平均損失={avg_val_loss:.4f}, 正解率={avg_val_accuracy:.4f}")

end_time_mp = time.time()
print(f"\n平均プーリングモデルのファインチューニングが完了しました。所要時間: {end_time_mp - start_time_mp:.2f} 秒")

# 5. 最終評価
print("\n--- 最終評価 (開発セット、平均プーリングモデル) ---")
model_meanpool.eval()
final_predictions_mp = []
final_true_labels_mp = []
with torch.no_grad():
    for batch in dev_dataloader_mp:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # token_type_ids = batch.get('token_type_ids', None)
        # if token_type_ids is not None:
        #     token_type_ids = token_type_ids.to(device)

        model_inputs_final_eval = {'input_ids': input_ids, 'attention_mask': attention_mask}
        # if token_type_ids is not None:
        #     model_inputs_final_eval['token_type_ids'] = token_type_ids
        logits = model_meanpool(**model_inputs_final_eval)
        
        predictions = torch.argmax(logits, dim=-1)
        
        final_predictions_mp.extend(predictions.cpu().numpy())
        final_true_labels_mp.extend(labels.cpu().numpy())

final_accuracy_mp = accuracy_score(final_true_labels_mp, final_predictions_mp)
print(f"開発セットにおける最終正解率 (平均プーリングモデル): {final_accuracy_mp:.4f}")
print(f"  正解した事例数: {int(final_accuracy_mp * len(final_true_labels_mp))}")
print(f"  総事例数: {len(final_true_labels_mp)}")

エラー: 前提となる変数 'train_bert_dataset' が定義されていません。


NameError: Variable 'train_bert_dataset' is not defined or None.

## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [ ]:
import os
# from openai import AzureOpenAI # 問題90のセルで初期化済みのはず
# from dotenv import load_dotenv # 問題90のセルで実行済みのはず

# --- 前提となる変数 ---
# client: 問題90で初期化された AzureOpenAI クライアントのインスタンス
# model_to_use: 問題94で使用したモデル名 (例: "gpt-4o")

# これらの変数が現在のセッションに存在することを確認してください。
if 'client' not in locals() or client is None:
    print("エラー: AzureOpenAIクライアント 'client' が初期化されていません。")
    print("問題90のセルを先に実行してクライアントを準備してください。")
    raise NameError("AzureOpenAI client 'client' is not defined.")
if 'model_to_use' not in locals() or model_to_use is None:
    # 問題94で使用したモデル名をここで設定するか、問題94のセルで定義してください
    model_to_use = "gpt-4o" # デフォルトまたは前回使用したモデル名
    print(f"警告: 'model_to_use' が未定義でしたので、'{model_to_use}' を使用します。")


# --- ここから問題95の処理 ---

# 1. 問題94のやり取りと、問題95の新しい質問を準備
# !!! 重要: assistant_response_94 は、実際に問題94を実行して得られたLLMの応答に置き換えてください !!!
# もし問題94をまだ実行していない場合、まず問題94を実行してこの応答を取得してください。
# 以下はダミーの応答です。必ず実際の応答で上書きしてください。
user_query_94 = "What do you call a sweet eaten after dinner?"
assistant_response_94 = "That's typically called dessert." # ← ★★★ 必ず問題94の実際のLLM応答に置き換えてください ★★★

if assistant_response_94 == "That's typically called dessert.": # ダミーのままなら警告
    print("警告: 'assistant_response_94' がダミーのままです。問題94を実行して実際のLLMの応答に置き換えてください。")
    print("このまま実行すると、LLMはダミーの応答を文脈として扱います。")


user_query_95 = "Please give me the plural form of the word with its spelling in reverse order."
system_prompt_95 = "You are a helpful assistant that answers questions precisely and follows instructions." # 例

# 2. messages リストの構築
messages_for_model_95 = [
    {"role": "system", "content": system_prompt_95},
    {"role": "user", "content": user_query_94},
    {"role": "assistant", "content": assistant_response_94}, # 問題94のLLMの応答
    {"role": "user", "content": user_query_95}        # 問題95の新しい質問
]

print("--- 言語モデルに与える messages リスト (プロンプト全体) ---")
for message in messages_for_model_95:
    print(message)

# 3. 言語モデルによる応答生成
try:
    print(f"\n--- モデル '{model_to_use}' からの応答生成中 (問題95)... ---")
    completion_95 = client.chat.completions.create(
        model=model_to_use,
        messages=messages_for_model_95,
        temperature=0.5, # 少し創造性を抑えめに (適宜調整)
        max_tokens=80    # 応答の最大長 (適宜調整)
    )
    
    # 4. 応答の表示
    if completion_95.choices:
        assistant_response_95 = completion_95.choices[0].message.content
        print("\nモデルの応答 (問題95):")
        print(assistant_response_95)
    else:
        print("モデルから応答が得られませんでした。")

except Exception as e:
    print(f"API呼び出し中にエラーが発生しました: {e}")

## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

In [ ]:
import os
import pandas as pd
import time
# from openai import AzureOpenAI # 問題90のセルで初期化済みのはず
from sklearn.metrics import accuracy_score # 正解率計算用

# --- 前提となる変数 ---
# client: 問題90で初期化された AzureOpenAI クライアントのインスタンス
# model_to_use: 使用するモデル名 (例: "gpt-4o")
# dev_file_path: SST-2のdev.tsvへのパス (問題60, 71, 85で定義済み)

# --- 変数・クラスが現在のセッションに存在するか確認 ---
if 'client' not in locals() or client is None:
    print("エラー: AzureOpenAIクライアント 'client' が初期化されていません。")
    raise NameError("AzureOpenAI client 'client' is not defined.")
if 'model_to_use' not in locals() or model_to_use is None:
    model_to_use = "gpt-4o" 
    print(f"警告: 'model_to_use' が未定義でしたので、'{model_to_use}' を使用します。")

base_data_dir_p96 = '../data/SST-2_data' # 問題85, 60などで使用したパスを想定
dev_file_path_p96 = os.path.join(base_data_dir_p96, "SST-2/dev.tsv")

if not os.path.exists(dev_file_path_p96):
    print(f"エラー: 検証データファイル '{dev_file_path_p96}' が見つかりません。")
    print("問題60を先に実行してSST-2データセットをダウンロード・展開してください。")
    raise FileNotFoundError("SST-2 dev.tsv not found.")

# --- ここから問題96の処理 ---

# 1. SST-2 開発データの読み込み
try:
    df_dev = pd.read_csv(dev_file_path_p96, sep='\t')
    print(f"検証データ ({os.path.basename(dev_file_path_p96)}) を読み込みました。件数: {len(df_dev)}")
except Exception as e:
    print(f"検証データの読み込み中にエラー: {e}")
    raise

def get_sentiment_from_llm(text_to_analyze, llm_client, model_name):
    """LLMにテキストの感情を問いかけ、応答から感情ラベルを抽出する"""
    prompt_template = """以下の映画レビューの感情は「ポジティブ」ですか、それとも「ネガティブ」ですか？ 回答は「ポジティブ」または「ネガティブ」のどちらか一言でお願いします。

レビュー: "{text}"
感情:"""
    
    prompt = prompt_template.format(text=text_to_analyze)
    
    try:
        completion = llm_client.chat.completions.create(
            model=model_name,
            messages=[
                # {"role": "system", "content": "You are an expert sentiment classifier."}, # 必要に応じてシステムプロンプト追加
                {"role": "user", "content": prompt}
            ],
            temperature=0.0, # 再現性のため、できるだけ決定的な出力を得る
            max_tokens=10    # "ポジティブ" or "ネガティブ" と少しの周辺語で十分
        )
        response_text = completion.choices[0].message.content.strip().lower()

        # LLMの応答からラベルを抽出 (より頑健な方法も検討可能)
        if "ポジティブ" in response_text or "positive" in response_text:
            return 1 # ポジティブ
        elif "ネガティブ" in response_text or "negative" in response_text:
            return 0 # ネガティブ
        else:
            # print(f"  警告: 予期せぬ応答 '{response_text}'。判定不能として扱います。")
            return -1 # 判定不能
            
    except Exception as e:
        print(f"  API呼び出し中にエラーが発生 (テキスト: \"{text_to_analyze[:50]}...\"): {e}")
        return -1 # エラー時は判定不能

# 3. 各レビュー文に対する感情予測と正解率計算
true_labels = []
predicted_labels_by_prompt = []

# APIのレート制限を考慮し、また全件実行すると時間がかかるため、
# devデータの一部 (例: 先頭100件) で試すことを推奨します。
# 全件 (872件) 実行する場合は時間がかかることとAPIコストに注意してください。
num_dev_samples_to_process = len(df_dev) # 全件処理する場合
# num_dev_samples_to_process = 100 # テスト用に件数を絞る場合

print(f"\nSST-2開発データの最初の{num_dev_samples_to_process}件について、プロンプトベースの感情分析を実行します...")

for index, row in df_dev.head(num_dev_samples_to_process).iterrows():
    text = str(row['sentence'])
    true_label = int(row['label'])
    
    predicted_label = get_sentiment_from_llm(text, client, model_to_use)
    
    if predicted_label != -1: # 判定不能・エラーでなかった場合のみ
        true_labels.append(true_label)
        predicted_labels_by_prompt.append(predicted_label)
    
    if (index + 1) % 20 == 0 or (index + 1) == num_dev_samples_to_process:
        print(f"  処理中: {index + 1}/{num_dev_samples_to_process} 件完了...")
    
    time.sleep(0.1) # APIのレート制限を避けるための短い待機 (必要に応じて調整)


# 4. 正解率の計算
if true_labels: # 有効な予測が1つでもあれば
    accuracy = accuracy_score(true_labels, predicted_labels_by_prompt)
    print(f"\n--- プロンプトによる感情分析の結果 (開発データ) ---")
    print(f"評価した事例数: {len(true_labels)}")
    print(f"正解率: {accuracy:.4f} ({accuracy*100:.2f}%)")
else:
    print("\n有効な予測が得られなかったため、正解率を計算できませんでした。")

## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import time
import os
import pandas as pd # 万が一の再読み込み用

# --- 前提となる変数・データ (問題85から) ---
# train_bert_dataset: 問題85で作成した訓練データ {'input_ids': tensor, 'attention_mask': tensor, 'labels': tensor}
# dev_bert_dataset: 問題85で作成した検証データ {'input_ids': tensor, 'attention_mask': tensor, 'labels': tensor}
# MAX_LENGTH: 問題85で使用した最大シーケンス長

# --- 変数・クラスが現在のセッションに存在するか確認 ---
# (問題87と同様の変数チェックをここに記述)
required_vars_p97 = ['train_bert_dataset', 'dev_bert_dataset', 'MAX_LENGTH']
for var_name in required_vars_p97:
    if var_name not in locals() or locals()[var_name] is None:
        print(f"エラー: 前提となる変数 '{var_name}' が定義されていません。")
        raise NameError(f"Variable '{var_name}' is not defined or None.")

# --- ここから問題97の処理 ---

# 1. モデルとトークナイザの準備
model_name_gpt_cls = "distilgpt2" # GPT-2系の比較的小さなモデルを例として使用
# model_name_gpt_cls = "gpt2" # もしリソースがあればgpt2も試せる

try:
    tokenizer_gpt_cls = AutoTokenizer.from_pretrained(model_name_gpt_cls)
    # GPT-2系トークナイザは通常パディングトークンが定義されていないので、EOSトークンで代用
    if tokenizer_gpt_cls.pad_token is None:
        tokenizer_gpt_cls.pad_token = tokenizer_gpt_cls.eos_token
        print(f"トークナイザ '{model_name_gpt_cls}' の pad_token を eos_token に設定しました。")
    
    # num_labels=2 で2値分類用のヘッドを持つモデルをロード
    # GPT-2系は pad_token_id も config に設定する必要がある場合がある
    model_gpt_cls = AutoModelForSequenceClassification.from_pretrained(
        model_name_gpt_cls, 
        num_labels=2,
        pad_token_id=tokenizer_gpt_cls.pad_token_id # pad_token_idを設定
    )
    print(f"事前学習済みモデル '{model_name_gpt_cls}' (分類用) をロードしました。")
    
    # GPT-2系モデルの分類ヘッドは、通常シーケンスの最後のトークンの隠れ状態を使うように設定されている。
    # そのため、トークナイザのパディング方向も重要になることがある。
    # tokenizer_gpt_cls.padding_side = 'left' # モデルによっては左パディングが良い場合も (今回はデフォルトの右を使用)

except Exception as e:
    print(f"モデル '{model_name_gpt_cls}' またはトークナイザのロード中にエラーが発生しました: {e}")
    raise

# 2. データセットとデータローダーの準備 (問題87のBertSST2Datasetを流用)
class GPTClassificationDataset(Dataset): # 問題87のBertSST2Datasetと同様の構造
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True, # GPT-2も通常は特殊トークンを期待 (入力形式による)
            max_length=self.max_len,
            return_token_type_ids=False, # GPT-2では通常不要
            padding='max_length',        # DataLoaderでバッチ化するために最大長にパディング
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long) # CrossEntropyLossはlong型のラベルを期待
        }

# SST-2の元のテキストとラベルを再度取得する必要がある
# (train_bert_datasetは既にID化されているため、ここでは元のDataFrameを想定)
# 問題60, 85で読み込んだdf_train, df_dev を使う
base_data_dir_p97 = '../data/SST-2_data' 
train_file_path_p97 = os.path.join(base_data_dir_p97, "SST-2/train.tsv")
dev_file_path_p97 = os.path.join(base_data_dir_p97, "SST-2/dev.tsv")

try:
    df_train_p97 = pd.read_csv(train_file_path_p97, sep='\t')
    df_dev_p97 = pd.read_csv(dev_file_path_p97, sep='\t')
except FileNotFoundError:
    print("エラー: SST-2のTSVファイルが見つかりません。問題60を再実行してください。")
    raise

train_texts_p97 = df_train_p97['sentence'].tolist()
train_labels_p97 = df_train_p97['label'].tolist()
dev_texts_p97 = df_dev_p97['sentence'].tolist()
dev_labels_p97 = df_dev_p97['label'].tolist()


train_dataset_gpt_cls = GPTClassificationDataset(train_texts_p97, train_labels_p97, tokenizer_gpt_cls, MAX_LENGTH)
dev_dataset_gpt_cls = GPTClassificationDataset(dev_texts_p97, dev_labels_p97, tokenizer_gpt_cls, MAX_LENGTH)

batch_size_gpt_cls = 16 
train_dataloader_gpt_cls = DataLoader(train_dataset_gpt_cls, batch_size=batch_size_gpt_cls, shuffle=True)
dev_dataloader_gpt_cls = DataLoader(dev_dataset_gpt_cls, batch_size=batch_size_gpt_cls, shuffle=False)


# 3. 学習の準備
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"\nGPU ({torch.cuda.get_device_name(0)}) を使用します。")
else:
    device = torch.device("cpu")
    print("\nGPUが利用できません。CPUを使用します。")

model_gpt_cls.to(device)

optimizer_gpt_cls = optim.AdamW(model_gpt_cls.parameters(), lr=5e-5) # GPT系のファインチューニングではBERTより少し高めの学習率も試される
num_epochs_gpt_cls = 3
total_steps_gpt_cls = len(train_dataloader_gpt_cls) * num_epochs_gpt_cls
scheduler_gpt_cls = get_linear_schedule_with_warmup(optimizer_gpt_cls, num_warmup_steps=0, num_training_steps=total_steps_gpt_cls)
# 損失関数はモデル内部で計算される (CrossEntropyLoss相当)

print(f"\nGPT系モデルのファインチューニング (バッチサイズ={batch_size_gpt_cls}, エポック数={num_epochs_gpt_cls}) を開始します...")
start_time_gpt_cls = time.time()

# 4. ファインチューニング（学習ループ） - 問題87とほぼ同じ
for epoch in range(num_epochs_gpt_cls):
    print(f"\n--- Epoch {epoch+1}/{num_epochs_gpt_cls} ---")
    model_gpt_cls.train()
    total_train_loss = 0
    
    for batch_idx, batch in enumerate(train_dataloader_gpt_cls):
        optimizer_gpt_cls.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model_gpt_cls(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        total_train_loss += loss.item()
        
        loss.backward()
        optimizer_gpt_cls.step()
        scheduler_gpt_cls.step()
        
        if (batch_idx + 1) % (len(train_dataloader_gpt_cls) // 10) == 0 or (batch_idx + 1) == len(train_dataloader_gpt_cls):
             print(f"  Batch [{batch_idx+1}/{len(train_dataloader_gpt_cls)}], Avg Train Loss so far: {total_train_loss / (batch_idx+1):.4f}")

    avg_train_loss = total_train_loss / len(train_dataloader_gpt_cls)
    print(f"Epoch [{epoch+1}/{num_epochs_gpt_cls}] 完了, 平均訓練損失: {avg_train_loss:.4f}")

    model_gpt_cls.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    with torch.no_grad():
        for batch in dev_dataloader_gpt_cls:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model_gpt_cls(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            total_eval_accuracy += accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy()) * labels.size(0)

    avg_val_accuracy = total_eval_accuracy / len(dev_dataset_gpt_cls.dataset) # Datasetの長さを分母に
    avg_val_loss = total_eval_loss / len(dev_dataloader_gpt_cls)
    print(f"  Epoch [{epoch+1}/{num_epochs_gpt_cls}], 検証データ: 平均損失={avg_val_loss:.4f}, 正解率={avg_val_accuracy:.4f}")

end_time_gpt_cls = time.time()
print(f"\nGPT系モデルのファインチューニングが完了しました。所要時間: {end_time_gpt_cls - start_time_gpt_cls:.2f} 秒")

# 5. 最終評価
print("\n--- 最終評価 (開発セット、GPT系分類モデル) ---")
model_gpt_cls.eval()
final_predictions_gpt_cls = []
final_true_labels_gpt_cls = []
with torch.no_grad():
    for batch in dev_dataloader_gpt_cls:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model_gpt_cls(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        final_predictions_gpt_cls.extend(predictions.cpu().numpy())
        final_true_labels_gpt_cls.extend(labels.cpu().numpy())

final_accuracy_gpt_cls = accuracy_score(final_true_labels_gpt_cls, final_predictions_gpt_cls)
print(f"開発セットにおける最終正解率 (GPT系分類モデル): {final_accuracy_gpt_cls:.4f}")
print(f"  正解した事例数: {int(final_accuracy_gpt_cls * len(final_true_labels_gpt_cls))}")
print(f"  総事例数: {len(final_true_labels_gpt_cls)}")

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

In [ ]:
import os
import pandas as pd
import json
import time
# from openai import AzureOpenAI # 問題90のセルで初期化済みのはず

# --- 前提となる変数 ---
# client: 問題90で初期化された AzureOpenAI クライアントのインスタンス
# model_to_use_for_ft_base: ファインチューニングのベースとなるモデル名 
#                           (例: "gpt-3.5-turbo-0125", Azureの場合はデプロイ名ではなくモデルバージョン名)
# train_file_path_p98: SST-2のtrain.tsvへのパス
# dev_file_path_p98: SST-2のdev.tsvへのパス

# --- 変数・クラスが現在のセッションに存在するか確認 ---
if 'client' not in locals() or client is None:
    print("エラー: AzureOpenAIクライアント 'client' が初期化されていません。")
    raise NameError("AzureOpenAI client 'client' is not defined.")

# ファインチューニングのベースモデル名 (Azureのモデルバージョン名か、OpenAIのモデルID)
# 例: Azureでgpt-3.5-turboのバージョン0125をファインチューニングする場合など
# OpenAI直接なら "gpt-3.5-turbo" など (利用可能なモデルを確認してください)
model_to_use_for_ft_base = "gpt-3.5-turbo-0125" # ★★★ ご自身の環境に合わせて変更 ★★★
                                            # Azureの場合、ファインチューニング可能なモデルを確認してください。
                                            # 例: gpt-35-turbo (バージョン指定が必要な場合あり)
                                            #      davinci-002, babbage-002 など (旧世代のモデル)

base_data_dir_p98 = '../data/SST-2_data' 
train_file_path_p98 = os.path.join(base_data_dir_p98, "SST-2/train.tsv")
dev_file_path_p98 = os.path.join(base_data_dir_p98, "SST-2/dev.tsv")

if not (os.path.exists(train_file_path_p98) and os.path.exists(dev_file_path_p98)):
    print(f"エラー: SST-2のTSVファイルが見つかりません。")
    raise FileNotFoundError("SST-2 TSV files not found.")

# --- ここから問題98の処理 ---

# 1. 学習データの準備 (JSONL形式)
def create_finetuning_data_jsonl(input_tsv_path, output_jsonl_path):
    """SST-2のTSVからファインチューニング用JSONLファイルを作成"""
    df = pd.read_csv(input_tsv_path, sep='\t')
    records = []
    prompt_prefix = "以下の映画レビューの感情は「ポジティブ」ですか、それとも「ネガティブ」ですか？ 回答は「ポジティブ」または「ネガティブ」のどちらか一言でお願いします。\n\nレビュー: "
    prompt_suffix = "\n感情:"
    
    for index, row in df.iterrows():
        sentence = str(row['sentence'])
        label = int(row['label'])
        
        completion_text = " ポジティブ" if label == 1 else " ネガティブ" # 先頭にスペース
        
        prompt_full = f"{prompt_prefix}{sentence}{prompt_suffix}"
        records.append({"prompt": prompt_full, "completion": completion_text})
        # OpenAIの新しいChat Completions Fine-tuning形式では "messages" を使う
        # records.append({"messages": [
        #     {"role": "system", "content": "You are a sentiment classifier. Respond with 'ポジティブ' or 'ネガティブ'."},
        #     {"role": "user", "content": f"レビュー: {sentence}\n感情:"},
        #     {"role": "assistant", "content": completion_text.strip()} # assistantの回答はスペースなし
        # ]})


    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for record in records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')
    print(f"ファインチューニング用データ '{output_jsonl_path}' を作成しました。 ({len(records)}件)")
    return output_jsonl_path

# 学習用JSONLファイルを作成 (SST-2訓練データの一部で試すことを推奨)
# 例: df_train.head(100) などで件数を絞る
# 全件 (約6万件) で行うと時間とコストがかかります
# ここでは、動作確認のため、非常に小さいダミーデータで例示します。
# 実際には train_file_path_p98 を使ってください。
# ---- ダミーデータ作成の例 (実際にはSST-2 train.tsvを使う) ----
dummy_train_data = {
    'sentence': [
        "this movie is fantastic and amazing", 
        "a truly boring and dull experience",
        "quite good, I enjoyed it",
        "I would not recommend this to anyone"
    ],
    'label': [1, 0, 1, 0]
}
df_dummy_train = pd.DataFrame(dummy_train_data)
dummy_train_tsv_path = "dummy_train_for_ft.tsv"
df_dummy_train.to_csv(dummy_train_tsv_path, sep='\t', index=False)
# ---- ダミーデータ作成ここまで ----

# 実際に使うファイルパス (最初はダミーで、慣れたら本番データで)
# training_file_jsonl_path = create_finetuning_data_jsonl(train_file_path_p98, "sst2_train_ft.jsonl")
training_file_jsonl_path = create_finetuning_data_jsonl(dummy_train_tsv_path, "dummy_sst2_train_ft.jsonl")


# 2. 学習ファイルのアップロード (OpenAI APIの例)
# Azure OpenAI の場合は、アップロード方法やAPI呼び出しが若干異なる可能性があります。
# Azure のドキュメントを参照してください。
uploaded_file_id = None
if os.path.exists(training_file_jsonl_path):
    try:
        print(f"\n学習ファイル '{training_file_jsonl_path}' をアップロードしています...")
        with open(training_file_jsonl_path, "rb") as f:
            # OpenAI Python v1.x.x 以降のファイルアップロード
            response = client.files.create(file=f, purpose="fine-tune")
            uploaded_file_id = response.id
        print(f"ファイルアップロード完了。File ID: {uploaded_file_id}")
    except Exception as e:
        print(f"ファイルアップロード中にエラー: {e}")
else:
    print(f"エラー: 学習ファイル '{training_file_jsonl_path}' が見つかりません。")

# 3. ファインチューニングジョブの作成
fine_tuned_model_id = None # ファインチューニング後のモデルIDを格納
if uploaded_file_id:
    try:
        print(f"\nファインチューニングジョブを作成しています... (ベースモデル: {model_to_use_for_ft_base}, File ID: {uploaded_file_id})")
        # OpenAI Python v1.x.x 以降
        # hyperparameters (エポック数など) も指定可能
        # suffix でカスタムモデル名に接尾辞を付けられる
        job = client.fine_tuning.jobs.create(
            training_file=uploaded_file_id,
            model=model_to_use_for_ft_base, # 例: "gpt-3.5-turbo-0125", "babbage-002", "davinci-002"
            # hyperparameters={"n_epochs": 1}, # エポック数など
            # suffix="sst2-sentiment" # カスタムモデル名の接尾辞
        )
        job_id = job.id
        print(f"ファインチューニングジョブ作成完了。Job ID: {job_id}")

        # 4. ファインチューニングジョブの監視
        print("\nジョブのステータスを監視します... (完了まで時間がかかります)")
        while True:
            job_status = client.fine_tuning.jobs.retrieve(job_id)
            status = job_status.status
            print(f"  現在のジョブステータス: {status} (Job ID: {job_id})")
            if status == "succeeded":
                fine_tuned_model_id = job_status.fine_tuned_model
                print(f"ファインチューニング成功！ ファインチューニング済みモデルID: {fine_tuned_model_id}")
                break
            elif status in ["failed", "cancelled"]:
                print(f"ファインチューニング失敗またはキャンセル。理由: {job_status.error}")
                break
            time.sleep(60) # 60秒ごとにステータス確認

    except Exception as e:
        print(f"ファインチューニングジョブの作成または監視中にエラー: {e}")

# 5. ファインチューニング済みモデルの利用 (検証データで評価)
if fine_tuned_model_id:
    print(f"\n--- ファインチューニング済みモデル ({fine_tuned_model_id}) で検証データを評価 ---")
    df_dev_p98 = pd.read_csv(dev_file_path_p98, sep='\t')
    
    true_labels_ft_eval = []
    predicted_labels_ft_eval = []
    
    # 問題96の get_sentiment_from_llm を参考に、ファインチューニング済みモデルで予測
    # ただし、プロンプトは学習時と同じものを使用
    prompt_prefix_eval = "以下の映画レビューの感情は「ポジティブ」ですか、それとも「ネガティブ」ですか？ 回答は「ポジティブ」または「ネガティブ」のどちらか一言でお願いします。\n\nレビュー: "
    prompt_suffix_eval = "\n感情:"

    # APIのレート制限を考慮し、一部でテスト推奨
    num_eval_samples = len(df_dev_p98) # 全件評価
    # num_eval_samples = 20 # テスト用に絞る場合

    for index, row in df_dev_p98.head(num_eval_samples).iterrows():
        text = str(row['sentence'])
        true_label = int(row['label'])
        
        prompt_for_eval = f"{prompt_prefix_eval}{text}{prompt_suffix_eval}"
        
        try:
            # ファインチューニング済みモデルIDを指定してCompletion APIを呼び出す
            # ChatCompletion形式のファインチューニングなら ChatCompletion API を使う
            # 以前のCompletion形式のファインチューニングなら Completion API (client.completions.create)
            # ここでは ChatCompletion 形式を仮定 (ベースモデルがgpt-3.5-turboなどの場合)
            completion_eval = client.chat.completions.create(
                model=fine_tuned_model_id, # ★★★ ファインチューニング済みモデルID ★★★
                messages=[{"role": "user", "content": prompt_for_eval}],
                temperature=0.0,
                max_tokens=10 
            )
            response_text = completion_eval.choices[0].message.content.strip().lower()

            pred_label_val = -1
            if "ポジティブ" in response_text or "positive" in response_text:
                pred_label_val = 1
            elif "ネガティブ" in response_text or "negative" in response_text:
                pred_label_val = 0
            
            if pred_label_val != -1:
                true_labels_ft_eval.append(true_label)
                predicted_labels_ft_eval.append(pred_label_val)

        except Exception as e:
            print(f"  評価中のAPI呼び出しエラー (テキスト: \"{text[:30]}...\"): {e}")
        
        if (index + 1) % 20 == 0 or (index + 1) == num_eval_samples:
            print(f"  評価処理中: {index + 1}/{num_eval_samples} 件完了...")
        time.sleep(0.1) # レート制限対策

    if true_labels_ft_eval:
        accuracy_ft_eval = accuracy_score(true_labels_ft_eval, predicted_labels_ft_eval)
        print(f"\nファインチューニング済みモデルの検証データにおける正解率: {accuracy_ft_eval:.4f} ({accuracy_ft_eval*100:.2f}%)")
        print(f"  評価した事例数: {len(true_labels_ft_eval)}")
    else:
        print("\n有効な予測が得られず、正解率を計算できませんでした。")
else:
    print("\nファインチューニング済みモデルIDが得られなかったため、評価をスキップします。")

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。


In [14]:
!uv pip install transformers torch datasets trl peft accelerate bitsandbytes

Using Python 3.11.10 environment at: /Users/ryuichi/.venv
⠙ Resolving dependencies...                                                     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 51 packages in 676ms                                        
Prepared 20 packages in 13.98s                                           
Uninstalled 1 package in 24ms
Installed 20 packages in 122ms                              
 + accelerate==1.6.0
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.11.18
 + aiosignal==1.3.2
 + bitsandbytes==0.42.0
 + datasets==3.6.0
 + dill==0.3.8
 + frozenlist==1.6.0
 - fsspec==2025.3.2
 + fsspec==2025.3.0
 + markdown-it-py==3.0.0
 + mdurl==0.1.2
 + multidict==6.4.3
 + multiprocess==0.70.16
 + peft==0.15.2
 + propcache==0.3.1
 + pyarrow==20.0.0
 + rich==14.0.0
 + trl==0.17.0
 + xxhash==3.5.0
 + yarl==1.20.0


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import DPOTrainer, DPOConfig
from peft import LoraConfig # LoRAを使う場合
import pandas as pd
import os

# --- 0. 基本設定と前提変数の確認 ---
# model_name_dpo_base = "gpt2"  # 例: GPT-2 (より高性能なオープンモデルを推奨)
# model_name_dpo_base = "distilgpt2" # 小さなモデルで試す場合
model_name_dpo_base = "EleutherAI/gpt-neo-125M" # より新しい小さめのオープンモデルの例

# SST-2データパス (問題85, 97などと同様)
base_data_dir_p99 = '../data/SST-2_data' 
train_file_path_p99 = os.path.join(base_data_dir_p99, "SST-2/train.tsv")
dev_file_path_p99 = os.path.join(base_data_dir_p99, "SST-2/dev.tsv")

if not os.path.exists(train_file_path_p99):
    print(f"エラー: 訓練データファイル '{train_file_path_p99}' が見つかりません。")
    raise FileNotFoundError("SST-2 train.tsv not found.")

# --- 1. モデルとトークナイザのロード ---
print(f"ベースモデル '{model_name_dpo_base}' とトークナイザをロード中...")
tokenizer_dpo = AutoTokenizer.from_pretrained(model_name_dpo_base)
if tokenizer_dpo.pad_token is None:
    tokenizer_dpo.pad_token = tokenizer_dpo.eos_token # GPT系ではよくある設定
    print("pad_token を eos_token に設定しました。")

# DPOでは通常、AutoModelForCausalLM を使う (生成モデルとして)
# LoRAを使う場合は、ここでベースモデルをロードし、後でPeftModelでラップする
# 量子化などを行う場合は、bitsandbytesの設定もここで行う
model_dpo = AutoModelForCausalLM.from_pretrained(model_name_dpo_base)
# ref_model = AutoModelForCausalLM.from_pretrained(model_name_dpo_base) # 参照モデル (DPOTrainerがNoneなら内部作成)
ref_model = None # DPOTrainerに内部作成を任せる場合

print("モデルとトークナイザのロード完了。")

# --- 2. 選好データセットの準備 ---
print("\n選好データセットを準備中...")
df_train_p99 = pd.read_csv(train_file_path_p99, sep='\t')

# データセットを大幅に削減してテスト (実際の学習では全件または十分な量を使用)
# df_train_p99 = df_train_p99.sample(n=1000, random_state=42) # 例: 1000件に削減
df_train_p99 = df_train_p99.head(100) # さらに小さくしてテスト

preference_data = []
prompt_template_dpo = "以下の映画レビューの感情を「ポジティブ」または「ネガティブ」で答えてください。\nレビュー: {review}\n感情:"

for _, row in df_train_p99.iterrows():
    review_text = str(row['sentence'])
    true_label = int(row['label'])
    
    prompt_filled = prompt_template_dpo.format(review=review_text)
    
    if true_label == 1: # ポジティブが望ましい
        chosen_completion = " ポジティブ" # 先頭にスペース
        rejected_completion = " ネガティブ"
    else: # ネガティブが望ましい
        chosen_completion = " ネガティブ"
        rejected_completion = " ポジティブ"
        
    preference_data.append({
        "prompt": prompt_filled,
        "chosen": chosen_completion, # モデルに生成させたい「良い」応答
        "rejected": rejected_completion # モデルに生成してほしくない「悪い」応答
    })

# Hugging Face Dataset形式に変換
# DPOTrainer は 'prompt', 'chosen', 'rejected' というキーを期待する
# また、入力はトークナイズされていないテキストで良い (DPOTrainerが内部でトークナイズ)
train_pref_dataset = Dataset.from_list(preference_data)
print(f"選好データセット準備完了。事例数: {len(train_pref_dataset)}")
print("選好データの例:")
print(train_pref_dataset[0])


# --- 3. (任意) PEFT (LoRA) の設定 ---
# LoRAを使うと、フルファインチューニングより少ない計算資源で済むことが多い
use_lora = True # LoRAを使用するかどうか
if use_lora:
    peft_config = LoraConfig(
        r=16,  # LoRAランク
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        # target_modules=['q_proj', 'v_proj'] # モデルによって対象モジュール名は異なる。自動検出も試みる。
        # target_modules="all-linear" # trlがサポートする便利な指定方法
    )
    print("\nLoRA設定を準備しました。")
else:
    peft_config = None


# --- 4. DPOTrainer の設定と学習 ---
# DPOConfig または TrainingArguments を使用
# DPOConfig は DPOTrainer に特化した引数を持ち、内部で TrainingArguments をラップする
output_dir_dpo = "./dpo_sentiment_model"

# DPOTrainer 用の設定 (trl 0.8.0 以降など、バージョンによって引数が変わる可能性あり)
# DPOConfigはTrainingArgumentsを継承しないので、両方設定するか、
# DPOTrainerにTrainingArgumentsを直接渡す古いスタイルもある。
# ここではDPOConfigを直接使う新しいスタイルを試みる（trlのバージョンに依存）

# まずはTrainingArgumentsで基本的な学習設定
# training_args = TrainingArguments(
#     per_device_train_batch_size=2, # GPUメモリに応じて調整
#     gradient_accumulation_steps=4, # 実質的なバッチサイズを増やす
#     learning_rate=1e-5, # DPOではSFTより少し高めも試される
#     num_train_epochs=1, # 選好学習は少ないエポックでも効果が出ることがある
#     logging_steps=10,
#     output_dir=output_dir_dpo,
#     # optim="adamw_torch", # 推奨
#     remove_unused_columns=False, # DPOTrainerが'prompt','chosen','rejected'以外の列を扱うため
#     # report_to="tensorboard", # tensorboardなどでログを見る場合
# )

# DPOTrainerの初期化 (trl 0.8.0以降を想定)
# 最新のtrlではDPOTrainerのコンストラクタ引数が変わっている可能性があるため、
# 公式ドキュメントを参照するのが最も確実です。
# beta, loss_type, max_prompt_length, max_length, max_target_length などが重要な引数。
try:
    dpo_trainer = DPOTrainer(
        model=model_dpo,
        ref_model=ref_model, # Noneにすると内部でコピーが作成される
        args=TrainingArguments( # DPOTrainerにはTrainingArgumentsを渡す
            output_dir=output_dir_dpo,
            per_device_train_batch_size=1, # メモリに応じて調整 (LoRAならもう少し増やせるかも)
            gradient_accumulation_steps=4,
            learning_rate=1.0e-5, # DPOでは比較的小さな学習率が良いとされることが多い
            num_train_epochs=1,   # DPOは少ないエポックで効果が出やすい
            logging_steps=10,
            remove_unused_columns=False, # 'prompt', 'chosen', 'rejected' を使うため
            # bf16=True, # 対応GPUならbf16で高速化・省メモリ化
            # report_to="none", # tensorboardなど使わない場合
        ),
        beta=0.1, # DPO損失のβパラメータ (0.1 ~ 0.5程度が多い)
        train_dataset=train_pref_dataset,
        tokenizer=tokenizer_dpo,
        peft_config=peft_config if use_lora else None, # LoRAを使う場合
        max_prompt_length=128, # プロンプトの最大長
        max_length=256,        # プロンプト＋生成の最大長 (max_prompt_length + max_target_length)
        # max_target_length=128 # 生成部分の最大長 (max_lengthからmax_prompt_lengthを引いたもの)
    )
    print("\nDPOTrainerを初期化しました。学習を開始します...")
    # 学習の実行
    dpo_trainer.train()
    print("DPO学習が完了しました。")

    # モデルの保存 (LoRAを使っている場合はアダプタのみ保存されることが多い)
    dpo_trainer.save_model(os.path.join(output_dir_dpo, "final_checkpoint"))
    tokenizer_dpo.save_pretrained(os.path.join(output_dir_dpo, "final_checkpoint"))
    print(f"学習済みモデルとトークナイザを '{os.path.join(output_dir_dpo, 'final_checkpoint')}' に保存しました。")
    
    # 学習後のモデルで評価 (問題96と同様のやり方で)
    # model_dpo (または dpo_trainer.model) を使って検証データで正解率を測定
    # (この部分は長くなるので骨子のみ)
    print("\n--- DPO学習後のモデルで検証データ評価 ---")
    # df_dev_p99 = pd.read_csv(dev_file_path_p99, sep='\t')
    # (問題96の get_sentiment_from_llm のような関数を、
    #  Hugging Faceモデルでテキスト生成するように修正して評価する)
    # 例:
    # model_to_eval = dpo_trainer.model # または AutoModelForCausalLM.from_pretrained でロード
    # model_to_eval.eval()
    # model_to_eval.to(device) # GPU使うなら
    # ... (devデータの各文でプロンプト作成 -> model.generate() -> 応答抽出 -> ラベル比較 -> 正解率計算) ...
    print("（評価部分は別途実装が必要です。問題96のプロンプトベース評価や、")
    print(" 問題98のファインチューニング後評価のロジックを参考に、")
    print(" Hugging Faceモデルでのテキスト生成と応答解釈を行ってください。）")


except ImportError as e:
    print(f"ImportError: {e}. 必要なライブラリ (trl, peft, accelerateなど) がインストールされているか確認してください。")
    print("`pip install trl peft accelerate bitsandbytes` を試してください。")
except Exception as e:
    print(f"DPOTrainerの初期化または学習中にエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()